# <h1 align="center"><font color="gree">Pydantic.AI</font></h1>

<font color="pink">Senior Data Scientist.: Dr. Eddy Giusepe Chirinos Isidro</font>

Este notebook foi baseado no tutorial de [Plaban Nayak](https://medium.com/the-ai-forum/building-an-agentic-system-to-enhance-rag-with-self-grading-and-web-search-capabilities-using-3f9a1d885730).

![](https://miro.medium.com/v2/resize:fit:4800/format:webp/1*I-kwOIjL3WrD3NL0WKLcYQ.jpeg)

In [1]:
from pydantic_ai import Agent, ModelRetry
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.models.groq import GroqModel
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


import nest_asyncio
nest_asyncio.apply()

import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
Eddy_key_openai  = os.environ['OPENAI_API_KEY']
Eddy_key_groq = os.environ['GROQ_API_KEY']
Eddy_key_gemini = os.environ['GEMINI_API_KEY']

In [2]:
openai_model = OpenAIModel('gpt-4o-mini')
groq_model = GroqModel("llama-3.3-70b-versatile")

In [ ]:
agent = Agent(
    'gemini-1.5-flash',
    system_prompt='Seja muito conciso, responda com uma frase apenas.',
    retries=3
)

result = agent.run_sync('Onde "hello world" vem?')

result.data

In [4]:
agent.model = 'gemini-1.5-pro'

@agent.system_prompt
async def get_system_prompt(self) -> str:
    return "Gere uma resposta longa e densa"

In [ ]:
from IPython.display import Markdown

result = agent.run_sync('Onde "hello world" vem?')
Markdown(result.data)

# <font color="red">Basic structured output</font>

In [7]:
from pydantic import BaseModel
from pydantic_ai import Agent

class MyModel(BaseModel):
    city: str
    country: str
    reason: str
    famous_person_from_city: str


model = 'openai:gpt-4o'

agent = Agent(model,
              result_type=MyModel,
              )


result = agent.run_sync('A cidade ventosa nos EUA.')

In [ ]:
result

In [ ]:
result.data

In [ ]:
result = agent.run_sync('A cidade do Merlion.')

result.data

In [ ]:
# UM exemplo de meu amado Perú:
result = agent.run_sync('A cidade onde está o Machu Picchu')

result.data

In [ ]:
result.data.famous_person_from_city

In [ ]:
agent

# <font color="red">Chat APP</font>


In [ ]:
from pydantic_ai import Agent
from pprint import pprint


agent = Agent('openai:gpt-4o-mini',
              system_prompt="""Seja um assistente útil e amigável.
                            """,
              retries=2,
              result_type=str
             )

result = agent.run_sync('Me conte uma piada.')

result.data

In [ ]:
# Todos os messages da execução:
pprint(result.all_messages())

# <font color="red">Chatting LLMs</font>

In [ ]:
from pydantic_ai import Agent

agent = Agent(model='openai:gpt-4o-mini',
              system_prompt='Seja um assistente útil e amigável.',
              retries=3,
              result_type=str
              )

result1 = agent.run_sync('Me conte uma piada.')
result1.data

result2 = agent.run_sync('Explique a piada.',
                        model='gemini-1.5-pro',
                        message_history=result1.new_messages()
                        )
result2.data


# <font color="red">Weather</font>

In [22]:
from __future__ import annotations as _annotations
import asyncio
import os
from dataclasses import dataclass
from typing import Any
from devtools import debug
from httpx import AsyncClient
from pydantic_ai import Agent, ModelRetry, RunContext
import logfire


@dataclass
class Deps:
    client: AsyncClient
    weather_api_key: str | None
    geo_api_key: str | None

In [23]:
weather_agent = Agent('openai:gpt-4o',
                      system_prompt='Seja conciso, responda com uma frase apenas.',
                      deps_type=Deps,
                      retries=2
                     )

@weather_agent.tool
async def get_lat_lng(
    ctx: RunContext[Deps], location_description: str
) -> dict[str, float]:
    """Encontre a latitude e longitude de uma localização.

    Args:
        ctx: O contexto.
        location_description: A descrição de uma localização.
    """
    if ctx.deps.geo_api_key is None:
        # Se não fornecido uma API key, retorne uma resposta fictícia (Londres):
        return {'lat': 51.1, 'lng': -0.1}

    params = {
        'q': location_description,
        'api_key': ctx.deps.geo_api_key,
    }
    with logfire.span('chamando API de geocodificação', params=params) as span:
        r = await ctx.deps.client.get('https://geocode.maps.co/search', params=params)
        r.raise_for_status()
        data = r.json()
        span.set_attribute('Resposta', data)

    if data:
        return {'lat': data[0]['lat'], 'lng': data[0]['lon']}
    else:
        raise ModelRetry('Não foi possível encontrar a localização')


@weather_agent.tool
async def get_weather(ctx: RunContext[Deps], lat: float, lng: float) -> dict[str, Any]:
    """Obtenha o clima em uma localização.

    Args:
        ctx: O contexto.
        lat: Latitude da localização.
        lng: Longitude da localização.
    """
    if ctx.deps.weather_api_key is None:
        # Se não fornecido uma API key, retorne uma resposta fictícia
        return {'temperature': '21 °C', 'description': 'Sunny'}

    params = {
        'apikey': ctx.deps.weather_api_key,
        'location': f'{lat},{lng}',
        'units': 'metric',
    }
    with logfire.span('chamando API de clima', params=params) as span:
        r = await ctx.deps.client.get(
            'https://api.tomorrow.io/v4/weather/realtime', params=params
        )
        r.raise_for_status()
        data = r.json()
        span.set_attribute('Resposta', data)

    values = data['data']['values']
    # https://docs.tomorrow.io/reference/data-layers-weather-codes
    code_lookup = {
        1000: 'Clear, Sunny',
        1100: 'Mostly Clear',
        1101: 'Partly Cloudy',
        1102: 'Mostly Cloudy',
        1001: 'Cloudy',
        2000: 'Fog',
        2100: 'Light Fog',
        4000: 'Drizzle',
        4001: 'Rain',
        4200: 'Light Rain',
        4201: 'Heavy Rain',
        5000: 'Snow',
        5001: 'Flurries',
        5100: 'Light Snow',
        5101: 'Heavy Snow',
        6000: 'Freezing Drizzle',
        6001: 'Freezing Rain',
        6200: 'Light Freezing Rain',
        6201: 'Heavy Freezing Rain',
        7000: 'Ice Pellets',
        7101: 'Heavy Ice Pellets',
        7102: 'Light Ice Pellets',
        8000: 'Thunderstorm',
    }
    return {
        'temperature': f'{values["temperatureApparent"]:0.0f}°C',
        'description': code_lookup.get(values['weatherCode'], 'Unknown'),
    }


In [24]:
async def main():
    async with AsyncClient() as client:
        # Crie uma chave API gratuita em https://www.tomorrow.io/weather-api/
        weather_api_key = os.getenv('WEATHER_API_KEY')
        # Crie uma chave API gratuita em https://geocode.maps.co/
        geo_api_key = os.getenv('GEO_API_KEY')
        deps = Deps(
            client=client, weather_api_key=weather_api_key, geo_api_key=geo_api_key
        )
        result = await weather_agent.run(
            'Qual é o clima em Londres e em Singapura?', deps=deps
        )
        debug(result)
        print('Resposta:', result.data)
        

In [ ]:
asyncio.run(main())

# <font color="red">Streaming MarkDown</font>

In [1]:
import asyncio
import nest_asyncio
nest_asyncio.apply()
import os

from rich.console import Console, ConsoleOptions, RenderResult
from rich.live import Live
from rich.markdown import CodeBlock, Markdown
from rich.syntax import Syntax
from rich.text import Text

from pydantic_ai import Agent
from pydantic_ai.models import KnownModelName


agent = Agent()

# Modelos para tentar, e a variável de ambiente apropriada:
models: list[tuple[KnownModelName, str]] = [
    ('gemini-1.5-flash', 'GEMINI_API_KEY'),
    ('openai:gpt-4o-mini', 'OPENAI_API_KEY'),
    ('groq:llama3-70b-8192', 'GROQ_API_KEY'),
]


async def main():
    prettier_code_blocks()
    console = Console()
    prompt = 'Mostre-me um exemplo curto de como usar Pydantic.'
    console.log(f'Perguntando: {prompt}...', style='cyan')
    for model, env_var in models:
        if env_var in os.environ:
            console.log(f'Usando modelo: {model}')
            with Live('', console=console, vertical_overflow='visible') as live:
                async with agent.run_stream(prompt, model=model) as result:
                    async for message in result.stream():
                        live.update(Markdown(message))
            #console.log(result.cost())
        else:
            console.log(f'{model} requer que {env_var} seja definido.')


def prettier_code_blocks():
    """Faça que os blocos de código do rich sejam mais bonitos e fáceis de copiar.

    De https://github.com/samuelcolvin/aicli/blob/v0.8.0/samuelcolvin_aicli.py#L22
    """

    class SimpleCodeBlock(CodeBlock):
        def __rich_console__(
            self, console: Console, options: ConsoleOptions
        ) -> RenderResult:
            code = str(self.text).rstrip()
            yield Text(self.lexer_name, style='dim')
            yield Syntax(
                code,
                self.lexer_name,
                theme=self.theme,
                background_color='black',
                word_wrap=True,
            )
            yield Text(f'/{self.lexer_name}', style='dim')

    Markdown.elements['fence'] = SimpleCodeBlock


if __name__ == '__main__':
    asyncio.run(main())

[23:31:45] Perguntando: Mostre-me um exemplo curto de como usar Pydantic....                       ]8;id=450562;file:///tmp/ipykernel_101622/3873954971.py\3873954971.py]8;;\:]8;id=711774;file:///tmp/ipykernel_101622/3873954971.py#30\30]8;;\

           Usando modelo: gemini-1.5-flash                                                         ]8;id=589669;file:///tmp/ipykernel_101622/3873954971.py\3873954971.py]8;;\:]8;id=309366;file:///tmp/ipykernel_101622/3873954971.py#33\33]8;;\

Output()

[23:31:49] Usando modelo: openai:gpt-4o-mini                                                       ]8;id=868991;file:///tmp/ipykernel_101622/3873954971.py\3873954971.py]8;;\:]8;id=384121;file:///tmp/ipykernel_101622/3873954971.py#33\33]8;;\

Output()

[23:31:55] Usando modelo: groq:llama3-70b-8192                                                     ]8;id=787783;file:///tmp/ipykernel_101622/3873954971.py\3873954971.py]8;;\:]8;id=794888;file:///tmp/ipykernel_101622/3873954971.py#33\33]8;;\

Output()